In [1]:
import pandas as pd

base = pd.read_csv('dataset.csv')

In [2]:
import numpy as np

previsores = base.iloc[:1400000,0:20].values
previsores = np.reshape(previsores, (2000, 700, 20))
print(previsores.shape)

classes = base.iloc[:1400000, 28:31].values
classes = np.reshape(classes, (2000, 700, 3))
print(classes.shape)

(2000, 700, 20)
(2000, 700, 3)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, CuDNNGRU, Bidirectional, Masking

In [5]:
def criarRede():
    model = Sequential()
  
    #model.add(Masking(mask_value = 0, input_shape = (700, 20)))
  
    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True), input_shape = (700, 20)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
    model.add(Dropout(0.3))

    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
    model.add(Dropout(0.3))

    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
    model.add(Dropout(0.3))
  
    model.add(Dense(3, activation = 'softmax'))
  
    model.compile(optimizer = 'adam', metrics = ['acc'], loss='categorical_crossentropy')
  
    return model

In [6]:
accu = []
precisions = []
recalls = []
f1 = []

In [7]:
from sklearn.metrics import balanced_accuracy_score, classification_report, precision_score, recall_score, f1_score, confusion_matrix

def train_and_evaluate_model(model, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train, epochs = 100, verbose = 1, batch_size = 32)
  
    predicted = model.predict(x_test)
  
    y_teste = []
    predict = []
  
    predicted = np.reshape(predicted, (predicted.shape[0] * predicted.shape[1], 3))
    y_test = np.reshape(y_test, (y_test.shape[0] * y_test.shape[1], 3))
    x_test = np.reshape(x_test, (x_test.shape[0] * x_test.shape[1], 20))

    for i in range(len(x_test)):
        cont = 0
        for j in range(len(x_test[i])):
            cont += x_test[i][j]
        if cont != 0:
            y_teste.append(y_test[i])
            predict.append(predicted[i])
    
    y_teste = np.asarray(y_teste)
    predict = np.asarray(predict)

    predicted = predict
    y_test = y_teste

    predicted = np.argmax(predicted, axis=1)
    y_test = np.argmax(y_test, axis=1)

    accu.append(balanced_accuracy_score(y_test, predicted)) 
    precisions.append(precision_score(y_test, predicted, average = 'weighted'))
    recalls.append(recall_score(y_test, predicted, average = 'weighted'))
    f1.append(f1_score(y_test, predicted, average = 'weighted'))
    
    print(classification_report(y_test, predicted))
    print('Acurácia')
    print(balanced_accuracy_score(y_test, predicted))
    print('Precisao')
    print(precision_score(y_test, predicted, average = 'weighted'))
    print('Recall')
    print(recall_score(y_test, predicted, average = 'weighted'))
    print('F1')
    print(f1_score(y_test, predicted, average = 'weighted'))
    print(confusion_matrix(y_test, predicted))

In [8]:
from sklearn.model_selection import KFold

kf = KFold(n_splits = 5, shuffle = True)

for train_index, test_index in kf.split(previsores):
    model = None
    model = criarRede()
    print("TRAIN:", train_index, "TEST:", test_index)
    train_and_evaluate_model(model, previsores[train_index], classes[train_index],
                           previsores[test_index], classes[test_index])

W0822 14:50:07.267557   504 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0822 14:50:07.272514   504 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0822 14:50:07.273512   504 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py

TRAIN: [   1    5    6 ... 1995 1998 1999] TEST: [   0    2    3    4    8   13   18   21   32   34   35   36   37   41
   44   48   52   53   60   66   69   73   74   78   81   98  107  116
  122  130  131  142  144  145  160  164  173  174  178  189  195  211
  213  221  226  227  229  234  236  239  240  247  248  249  250  251
  252  253  255  268  269  274  276  278  279  281  282  284  285  293
  300  304  312  322  329  332  349  351  353  356  363  366  372  376
  386  387  392  393  396  401  406  407  408  410  411  413  424  427
  440  445  451  452  453  462  473  474  475  479  481  489  493  500
  502  509  510  514  516  518  522  532  535  543  548  553  554  557
  563  566  569  570  572  577  584  588  595  598  604  605  612  616
  618  639  644  647  648  650  652  656  685  688  694  706  707  709
  712  713  721  725  727  728  736  740  742  743  744  746  747  748
  750  756  760  764  766  773  782  789  790  794  797  804  818  822
  830  836  840  841  845  8

W0822 14:50:08.787252   504 deprecation.py:323] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.3322 - acc: 0.8376
Epoch 2/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3077 - acc: 0.8614
Epoch 3/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3019 - acc: 0.8642
Epoch 4/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2996 - acc: 0.8650
Epoch 5/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2967 - acc: 0.8662
Epoch 6/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.2946 - acc: 0.8674
Epoch 7/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.2935 - acc: 0.8679
Epoch 8/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.2899 - acc: 0.8700
Epoch 9/100
1600/1600 [==============================] - 14s 9ms/sample - loss: 0.2888 - acc: 0.8697
Epoch 10/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2883 - 

1600/1600 [==============================] - 15s 10ms/sample - loss: 0.1449 - acc: 0.9394
Epoch 82/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1434 - acc: 0.9400
Epoch 83/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1421 - acc: 0.9405
Epoch 84/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1411 - acc: 0.9409
Epoch 85/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1415 - acc: 0.9408
Epoch 86/100
1600/1600 [==============================] - 14s 9ms/sample - loss: 0.1417 - acc: 0.9409
Epoch 87/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1400 - acc: 0.9413
Epoch 88/100
1600/1600 [==============================] - 14s 9ms/sample - loss: 0.1381 - acc: 0.9423
Epoch 89/100
1600/1600 [==============================] - 15s 10ms/sample - loss: 0.1374 - acc: 0.9429
Epoch 90/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1368 - acc

...

Epoch 1/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.3348 - acc: 0.7371
Epoch 2/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.3116 - acc: 0.8588
Epoch 3/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.3045 - acc: 0.8629
Epoch 4/100
1600/1600 [==============================] - 14s 9ms/sample - loss: 0.3043 - acc: 0.8623
Epoch 5/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2999 - acc: 0.8642
Epoch 6/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2981 - acc: 0.8651
Epoch 7/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2946 - acc: 0.8670
Epoch 8/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2947 - acc: 0.8667
Epoch 9/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2911 - acc: 0.8685
Epoch 10/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2891 -

1600/1600 [==============================] - 14s 9ms/sample - loss: 0.1462 - acc: 0.9388
Epoch 82/100
1600/1600 [==============================] - 14s 9ms/sample - loss: 0.1445 - acc: 0.9394
Epoch 83/100
1600/1600 [==============================] - 14s 9ms/sample - loss: 0.1437 - acc: 0.9398
Epoch 84/100
1600/1600 [==============================] - 14s 9ms/sample - loss: 0.1436 - acc: 0.9398
Epoch 85/100
1600/1600 [==============================] - 14s 9ms/sample - loss: 0.1418 - acc: 0.9407
Epoch 86/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1406 - acc: 0.9413
Epoch 87/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1408 - acc: 0.9410
Epoch 88/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1399 - acc: 0.9415
Epoch 89/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1391 - acc: 0.9418
Epoch 90/100
1600/1600 [==============================] - 15s 10ms/sample - loss: 0.1392 - acc:

1600/1600 [==============================] - 15s 10ms/sample - loss: 0.2240 - acc: 0.9036
Epoch 36/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2221 - acc: 0.9046
Epoch 37/100
1600/1600 [==============================] - 14s 9ms/sample - loss: 0.2195 - acc: 0.9053
Epoch 38/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2180 - acc: 0.9058
Epoch 39/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2151 - acc: 0.9077
Epoch 40/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2122 - acc: 0.9089
Epoch 41/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2108 - acc: 0.9090
Epoch 42/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2085 - acc: 0.9104
Epoch 43/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2052 - acc: 0.9121
Epoch 44/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2039 - acc

Epoch 1/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.3353 - acc: 0.8023
Epoch 2/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3113 - acc: 0.8597
Epoch 3/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3069 - acc: 0.8619
Epoch 4/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3025 - acc: 0.8638
Epoch 5/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2999 - acc: 0.8653
Epoch 6/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2984 - acc: 0.8660
Epoch 7/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2962 - acc: 0.8671
Epoch 8/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2928 - acc: 0.8688
Epoch 9/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2921 - acc: 0.8688
Epoch 10/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2903 - a

1600/1600 [==============================] - 13s 8ms/sample - loss: 0.1458 - acc: 0.9391
Epoch 82/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.1459 - acc: 0.9391
Epoch 83/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.1448 - acc: 0.9394
Epoch 84/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.1433 - acc: 0.9401
Epoch 85/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.1431 - acc: 0.9403
Epoch 86/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.1420 - acc: 0.9405
Epoch 87/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.1403 - acc: 0.9415
Epoch 88/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.1400 - acc: 0.9413
Epoch 89/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.1403 - acc: 0.9415
Epoch 90/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.1390 - acc: 

In [9]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_20 (Bidirectio (None, 700, 200)          73200     
_________________________________________________________________
dropout_20 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_21 (Bidirectio (None, 700, 200)          181200    
_________________________________________________________________
dropout_21 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_22 (Bidirectio (None, 700, 200)          181200    
_________________________________________________________________
dropout_22 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_23 (Bidirectio (None, 700, 200)         

In [11]:
print('Acurácias total')
print(accu)
accu = np.array(accu)
print(accu.mean())
print('Precision total')
print(precisions)
precisions = np.array(precisions)
print(precisions.mean())
print('Recalls total')
print(recalls)
recalls = np.array(recalls)
print(recalls.mean())
print('F1 total')
print(f1)
f1 = np.array(f1)
print(f1.mean())

Acurácias total
[0.70537274 0.70944676 0.70761891 0.71745467 0.70088234]
0.7081550810090278
Precision total
[0.71972733 0.72465629 0.71860687 0.72931164 0.71565051]
0.7215905283084029
Recalls total
[0.72077441 0.72221773 0.71789164 0.72775858 0.71605353]
0.7209391781612478
F1 total
[0.71940028 0.72221423 0.71771944 0.7280167  0.71530666]
0.720531463028926
